In [4]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import process_image, color_enclosed_black_areas

In [10]:
import cv2
import numpy as np
import math

def draw_line_with_angle(image):
    """
    Trova il punto bianco più basso appartenente a una linea nell'immagine e traccia una linea
    per tutta l'immagine con un angolo di 70 gradi rispetto alla linea a cui appartiene il punto.

    Args:
        image (numpy.ndarray): Immagine binaria (0 e 255).

    Returns:
        numpy.ndarray: Immagine con la linea tracciata.
    """
    image_with_line = image.copy()
    height, width = image.shape

    # Trova i contorni
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return image_with_line  # Nessun contorno trovato

    # Trova il punto bianco più basso tra tutti i contorni
    lowest_point = None
    for contour in contours:
        for point in contour:
            x, y = point[0]
            if lowest_point is None or y > lowest_point[1]:
                lowest_point = (x, y)

    if lowest_point is None:
        return image_with_line  # Nessun punto bianco trovato

    # Calcola la direzione della linea con un angolo di 70 gradi
    angle_rad = math.radians(70)  # Converti 70 gradi in radianti
    dx = math.cos(angle_rad)  # Delta x (variazione lungo l'asse x)
    dy = math.sin(angle_rad)  # Delta y (variazione lungo l'asse y)

    # Calcola i punti di inizio e fine della linea
    start_point = (int(lowest_point[0] - dx * height), int(lowest_point[1] - dy * height))
    end_point = (int(lowest_point[0] + dx * height), int(lowest_point[1] + dy * height))

    # Assicurati che i punti siano entro i limiti dell'immagine
    start_point = (max(0, min(width - 1, start_point[0])), max(0, min(height - 1, start_point[1])))
    end_point = (max(0, min(width - 1, end_point[0])), max(0, min(height - 1, end_point[1])))

    # Disegna la linea sull'immagine
    cv2.line(image_with_line, start_point, end_point, (255, 255, 255), thickness=5)

    return image_with_line

# Esempio di utilizzo
# image = cv2.imread('path_to_image', cv2.IMREAD_GRAYSCALE)
# binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1]
# result_image = draw_line_with_angle(binary_image)
# cv2.imshow('Result', result_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [5]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    return mask_cleaned

In [12]:
imageURL = "output/246072.png"
image = cv2.imread(imageURL)
image = preprocess_image(image)
image = draw_line_with_angle(image)
cv2.imshow("Image with Horizontal Lines", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def color_surrounded_areas(image):
    """
    Colora di rosso le aree nere completamente circondate dal bianco.

    Args:
        image (numpy.ndarray): Immagine binaria (0 e 255).

    Returns:
        numpy.ndarray: Immagine con le aree nere circondate colorate di rosso.
    """
    # Converti l'immagine in formato BGR per colorazione
    image_colored = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Trova i contorni
    contours, _ = cv2.findContours(image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    for i, contour in enumerate(contours):
        # Controlla se il contorno è interno (gerarchia RETR_CCOMP)
        if cv2.contourArea(contour) > 0:  # Ignora aree piccolissime
            # Disegna l'area interna di rosso se circondata
            cv2.drawContours(image_colored, [contour], -1, (0, 0, 255), thickness=cv2.FILLED)

    return image_colored

In [30]:
def find_midpoint_of_top_side(image):
    """
    Trova il punto medio del lato più alto di un'area rossa rettangolare nell'immagine.

    Args:
        image (numpy.ndarray): Immagine a colori (BGR).

    Returns:
        tuple: Coordinata (x, y) del punto medio del lato più alto, o None se non trovata.
    """
    # Crea una maschera per individuare le aree rosse (0, 0, 255)
    red_mask = cv2.inRange(image, (0, 0, 255), (0, 0, 255))

    # Trova i contorni delle aree rosse
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approssima il contorno a un rettangolo
        x, y, w, h = cv2.boundingRect(contour)
        top_midpoint = (x + w // 2, y)  # Punto medio del lato superiore
        # Calcola la retta perpendicolare
        perp_start = (top_midpoint[0], top_midpoint[1] - 50)  # 50 pixel sopra il punto medio
        perp_end = (top_midpoint[0], top_midpoint[1] + 50)  # 50 pixel sotto il punto medio

        return top_midpoint, perp_start, perp_end

    return None, None, None

In [39]:
def find_highest_segment_midpoint_and_perpendicular(mask):
    """
    Trova il segmento più alto in un'immagine binaria utilizzando la trasformata di Hough, calcola
    il punto medio e determina una retta perpendicolare al segmento.

    Args:
        mask (numpy.ndarray): Immagine binaria (0 e 255).

    Returns:
        tuple: Coordinata (x, y) del punto medio e punti di inizio e fine della retta perpendicolare.
    """
    # Trova linee con la trasformata di Hough
    lines = cv2.HoughLinesP(mask, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

    if lines is None:
        return None, None, None  # Nessuna linea trovata

    # Trova il segmento più alto
    highest_segment = None
    min_y = float('inf')

    for line in lines:
        x1, y1, x2, y2 = line[0]
        avg_y = (y1 + y2) / 2
        if avg_y < min_y:
            min_y = avg_y
            highest_segment = (x1, y1, x2, y2)

    if highest_segment is None:
        return None, None, None  # Nessun segmento valido trovato

    x1, y1, x2, y2 = highest_segment

    # Calcola il punto medio del segmento
    midpoint = ((x1 + x2) // 2, (y1 + y2) // 2)

    # Calcola la pendenza del segmento
    if x2 != x1:  # Evita divisioni per zero
        slope = (y2 - y1) / (x2 - x1)
    else:
        slope = float('inf')  # Segmento verticale

    # Calcola la pendenza della retta perpendicolare
    if slope != 0 and slope != float('inf'):
        perp_slope = -1 / slope
    else:
        perp_slope = 0 if slope == float('inf') else float('inf')

    # Calcola i punti di inizio e fine della retta perpendicolare
    length = 50  # Lunghezza della retta perpendicolare (metà sopra e metà sotto il punto medio)
    if perp_slope == float('inf'):
        perp_start = (midpoint[0], midpoint[1] - length)
        perp_end = (midpoint[0], midpoint[1] + length)
    elif perp_slope == 0:
        perp_start = (midpoint[0] - length, midpoint[1])
        perp_end = (midpoint[0] + length, midpoint[1])
    else:
        dx = int(length / math.sqrt(1 + perp_slope**2))
        dy = int(perp_slope * dx)
        perp_start = (midpoint[0] - dx, midpoint[1] - dy)
        perp_end = (midpoint[0] + dx, midpoint[1] + dy)

    return midpoint, perp_start, perp_end

In [50]:
def spline_cubica(p0, t0, p1, t1, num_points=100):
    """
    Calcola una spline cubica tra due punti con tangenti specificate.
    
    :param p0: Punto iniziale (x0, y0)
    :param t0: Tangente al punto iniziale (tx0, ty0)
    :param p1: Punto finale (x1, y1)
    :param t1: Tangente al punto finale (tx1, ty1)
    :param num_points: Numero di punti per disegnare la curva
    :return: Lista di punti della spline cubica
    """

     # Calcolare i vettori tangenti dalla differenza tra i punti p0, t0 e p1, t1
    t0 = np.array(t0) - np.array(p0)  # Vettore tangente iniziale
    t1 = np.array(t1) - np.array(p1)  # Vettore tangente finale
    t = np.linspace(0, 1, num_points)
    h00 = 2 * t**3 - 3 * t**2 + 1
    h10 = t**3 - 2 * t**2 + t
    h01 = -2 * t**3 + 3 * t**2
    h11 = t**3 - t**2

    spline_x = h00 * p0[0] + h10 * t0[0] + h01 * p1[0] + h11 * t1[0]
    spline_y = h00 * p0[1] + h10 * t0[1] + h01 * p1[1] + h11 * t1[1]

    return np.array(list(zip(spline_x, spline_y)), dtype=np.int32)

In [52]:
img = cv2.imread("output/246072.png")
img = preprocess_image(img)
# Trova i pixel bianchi
white_pixels = np.column_stack(np.where(img == 255))

# Se esistono pixel bianchi, trova il punto con la massima coordinata y
if len(white_pixels) > 0:
    lowest_point = white_pixels[np.argmax(white_pixels[:, 0])]
    y_lowest, x_lowest = lowest_point
    print(f"Punto bianco più basso: ({x_lowest}, {y_lowest})")
    
    # Calcola le coordinate della linea inclinata di 30 gradi
    length = 450  # Lunghezza della linea (modificabile)
    angle = math.radians(30)  # Inclinazione in radianti
    
    # Punto finale della linea
    x_end = int(x_lowest + length * math.cos(angle))
    y_end = int(y_lowest - length * math.sin(angle))  # y diminuisce verso l'alto

    # Disegna la linea sull'immagine originale
    output_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Converti in BGR per colori
    cv2.line(output_img, (x_lowest, y_lowest), (x_end, y_end), (255, 255, 255), thickness=8)  
    img = cv2.cvtColor(output_img, cv2.COLOR_BGR2GRAY)
    colored_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    contours, _ = cv2.findContours(255 - img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    height, width = img.shape
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        if x > 0 and y > 0 and (x + w) < width and (y + h) < height:
            if area > 1000: 
                print(area)
                cv2.drawContours(colored_image, [contour], -1, (0,0,255), thickness=cv2.FILLED)

    hsv_image = cv2.cvtColor(colored_image, cv2.COLOR_BGR2HSV)

# Definisci i range per il colore rosso in formato HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

# Maschera per isolare il colore rosso
    mask = cv2.inRange(hsv_image, lower_red, upper_red)
    mask = cv2.Canny(mask,100,200)

    midpoint, perp_start, perp_end = find_highest_segment_midpoint_and_perpendicular(mask)
    print(midpoint)
    cv2.circle(colored_image, midpoint, 5, (255, 0, 0), -1)
    curve_points = spline_cubica((400,600), (400,500), midpoint, perp_start)
    for i in range(1, len(curve_points)):
        cv2.line(colored_image, curve_points[i - 1], curve_points[i], (0, 255, 255), 1)

    cv2.circle(colored_image, (400,600), 5, (255, 255, 255), -1)  # Giallo
    cv2.circle(colored_image, midpoint, 5, (255, 255, 255), -1)  # Giallo
    cv2.imshow('Linea inclinata', colored_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Punto bianco più basso: (37, 395)
42339.0
(130, 133)
